## Word2Vec ## 

In this notebook we will go through the step by step creation of the Continouous Bag Of Words (CBOW).
CBOW is an embedded model that makes use of a "fake task" -> [within short window, predict the current word] to extract a vector that shows the relationship between words.

### Continuous Bag Of Words ###

Adapted from Robert Guthrie

In [1]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.linalg

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = "cpu"

In [3]:
# CBOW is a window view; we are trying to infer the word in the middle.
CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right

raw_text= """Long Short-Term Memory (LSTM) is a recurrent neural network (RNN) architecture that has been designed
to address the vanishing and exploding gradient problems of conventional RNNs. Unlike feedforward neural networks,
RNNs have cyclic connections making them powerful for modeling sequences. 
They have been successfully used for sequence labeling and sequence prediction tasks,
such as handwriting recognition, language modeling, phonetic labeling of acoustic frames. However, in contrast to the deep neural
networks, the use of RNNs in speech recognition has been limited to phone recognition in small scale tasks. 
In this paper, we present novel LSTM based RNN architectures which make more effective
use of model parameters to train acoustic models for large vocabulary speech recognition. 
We train and compare LSTM, RNN and DNN models at various numbers of parameters and configurations.
We show that LSTM models converge quickly and give state of the art speech recognition performance for relatively small sized models.""".split()

# By deriving a set from "raw_text", we deduplicate the array
vocab = set(raw_text)
vocab_size = len(vocab)

# Basic Tokenizer
word_to_ix = {word: i for i, word in enumerate(vocab)}

print(len(raw_text))
print(vocab_size)

152
106


In [4]:
# Now lets create a "dataset"
data = []
for i in range(CONTEXT_SIZE, len(raw_text) - CONTEXT_SIZE):
    context = []
    for j in range(CONTEXT_SIZE, 0, -1):
        context.append(raw_text[i - j])

    for j in range(1, CONTEXT_SIZE + 1):
        context.append(raw_text[i + j])
        
    target = raw_text[i]
    data.append((context, target))
print(data[:5])


[(['Long', 'Short-Term', '(LSTM)', 'is'], 'Memory'), (['Short-Term', 'Memory', 'is', 'a'], '(LSTM)'), (['Memory', '(LSTM)', 'a', 'recurrent'], 'is'), (['(LSTM)', 'is', 'recurrent', 'neural'], 'a'), (['is', 'a', 'neural', 'network'], 'recurrent')]


### Create the CBOW Model (as we have seen already other ANN) ###

We have to extend from nn.Module as all the other ANN

In [5]:
class CBOW(nn.Module):

    def __init__(self, vocab_size, embed_dim, context, hidden_size):
        super(CBOW, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.linear = nn.Sequential(
            nn.Linear(context*embed_dim, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, vocab_size),
            nn.LogSoftmax(dim = -1)
        )
        
    def forward(self, inputs):
#         print(inputs.shape)
#         print(inputs)
        out = self.embedding(inputs)
#         print(out.shape)
        out = out.view(1, -1)
#         print(out.shape)
        out = self.linear(out)
#         print(out.shape)
        return out
    
    # This is what we are actually interested on
    def get_word_vector(self, word):
        out = self.embedding(word)
        return out


#### Lets break it down! ####

In [6]:
VOCAB_SIZE = len(vocab)
EMBEDD_DIM = 10
BATCH_SIZE = 6
FULL_CONTEXT_SIZE = CONTEXT_SIZE * 2
HIDDEN_SIZE = 256

example_tensor = torch.randint(0, VOCAB_SIZE, [BATCH_SIZE, FULL_CONTEXT_SIZE])
print(example_tensor)

tensor([[ 15,  12,  11,  32],
        [ 52,  35,  10,  92],
        [ 14,  32,  65,  19],
        [ 48,  68,  33,  87],
        [ 69, 104,  60, 100],
        [ 59,  68,  86,  87]])


In [7]:
CBOW_embedding = nn.Embedding(VOCAB_SIZE, EMBEDD_DIM)
example_result = CBOW_embedding(example_tensor)
# Now we have a representation of the words in a vector of EMBEDD_DIM Dimensions
print(example_result.shape)
# example_result = torch.flatten(example_result, start_dim=1)
example_result = example_result.view(BATCH_SIZE, -1)
print(example_result.shape)

torch.Size([6, 4, 10])
torch.Size([6, 40])


In [8]:
CBOW_hidden = nn.Linear(EMBEDD_DIM * FULL_CONTEXT_SIZE, HIDDEN_SIZE)
CBOW_hidden_relu = nn.ReLU()
example_result = CBOW_hidden(example_result)
example_result = CBOW_hidden_relu(example_result)
print(example_result.shape)

torch.Size([6, 256])


In [9]:
CBOW_output = nn.Linear(HIDDEN_SIZE, VOCAB_SIZE)
CBOW_output_soft = nn.LogSoftmax(dim = -1)
example_result = CBOW_output(example_result)
example_result = CBOW_output_soft(example_result)
print(example_result.shape)

torch.Size([6, 106])


In [10]:
print(example_result[0].argmax(-1))
print(example_result[0])
print(example_result[1].argmax(-1))
print(example_result[1])
# print(example_result[2].argmax(-1))
# print(example_result[3].argmax(-1))
# print(example_result[4].argmax(-1))

tensor(35)
tensor([-4.5491, -4.4814, -4.7106, -4.9417, -4.6276, -4.8057, -4.5576, -4.5030,
        -4.5510, -4.4183, -4.5823, -4.1000, -4.4622, -4.3611, -4.8388, -4.7348,
        -4.4998, -4.9934, -4.7053, -4.7468, -5.1095, -5.1439, -4.5026, -4.6723,
        -4.7022, -4.4329, -4.2943, -4.7277, -4.7178, -4.4072, -4.5628, -4.6035,
        -4.3589, -5.1638, -4.5705, -3.9938, -5.0362, -4.2808, -4.7545, -4.8238,
        -4.7990, -4.5225, -4.6346, -5.2335, -4.7047, -4.5037, -4.9659, -5.3077,
        -4.4388, -4.5231, -5.1118, -4.5026, -4.4559, -4.8398, -4.9712, -4.6266,
        -5.1883, -5.1041, -4.6661, -4.8765, -4.7365, -4.7943, -4.7292, -4.5441,
        -5.0134, -4.2937, -4.6687, -4.9026, -4.6107, -4.6394, -4.9024, -4.8883,
        -4.6313, -4.6326, -4.8319, -4.7311, -5.0951, -4.7691, -4.2984, -4.6508,
        -4.7083, -4.8886, -5.0517, -4.7958, -4.3001, -4.7462, -4.7414, -4.7212,
        -4.8861, -4.9821, -4.8096, -4.7537, -4.3322, -4.2995, -4.7603, -5.0841,
        -4.9572, -4.4448, -4.

## Back to the notebook ##

In [11]:
# Simple helper method to transform the context to the expected int vector - tensor

def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)

make_context_vector(data[0][0], word_to_ix)

tensor([ 1, 22,  7, 93])

In [12]:
def train(model, epochs, data, optimizer, loss_fn):
    model.train()
    losses = []
    for epoch in range(epochs):
        total_loss = 0
        for context, target in data:

            # Prepare inputs and targets 
            context_idxs = make_context_vector(context, word_to_ix)
            context_idxs = context_idxs.to(device)
            target_id = make_context_vector([target], word_to_ix)
            target_id = target_id.to(device)

            # Do not accumulate 
            model.zero_grad()

            # Step 3. Run the forward pass
            log_probs = model(context_idxs)
    #         break

            # Step 4. Compute your loss function.
            loss = loss_fn(log_probs, target_id)

    #         loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))

            # Step 5. Do the backward pass and update the gradient
            loss.backward()
            optimizer.step()

            # Get the Python number from a 1-element Tensor by calling tensor.item()
            total_loss += loss.item()
        losses.append(total_loss)
    return losses
    

In [13]:
VOCAB_SIZE = len(vocab)
EMBEDD_DIM = 10
BATCH_SIZE = 6
FULL_CONTEXT_SIZE = CONTEXT_SIZE * 2
HIDDEN_SIZE = 256

loss_function = nn.NLLLoss() # Because we are using Log_softmax
model = CBOW(vocab_size, EMBEDD_DIM, FULL_CONTEXT_SIZE, HIDDEN_SIZE)
model = model.to(device)
optimizer = optim.SGD(model.parameters(), lr=0.001)

lossesCBOW = train(model, 100, data, optimizer, loss_function)
model.eval()

print(lossesCBOW)  # The loss decreased every iteration over the training data!

[698.8276858329773, 691.5202617645264, 684.3134832382202, 677.2004256248474, 670.1764786243439, 663.2351455688477, 656.3711779117584, 649.578705072403, 642.8536560535431, 636.191891670227, 629.5857443809509, 623.0297882556915, 616.5213692188263, 610.0574922561646, 603.6324412822723, 597.2438929080963, 590.8856863975525, 584.5560214519501, 578.2520091533661, 571.96653175354, 565.6991097927094, 559.4459638595581, 553.2070643901825, 546.9738887548447, 540.7462080717087, 534.5192048549652, 528.2936021089554, 522.0670284032822, 515.8377195596695, 509.60809099674225, 503.374799489975, 497.1363010406494, 490.8889070749283, 484.63085770606995, 478.3647960424423, 472.09297573566437, 465.8115938901901, 459.52144277095795, 453.2244362831116, 446.9176627397537, 440.60235273838043, 434.28279560804367, 427.9590892791748, 421.6355796456337, 415.3114558458328, 408.9858821630478, 402.66197884082794, 396.33923691511154, 390.01971477270126, 383.7030522823334, 377.39566272497177, 371.09751230478287, 364.8

In [14]:
# list out keys and values separately
key_list = list(word_to_ix.keys())
val_list = list(word_to_ix.values())

In [15]:
def similarity_cbow(word_1, word_2):
    
    # test word similarity
    print(word_1)
    print(word_2)
    w1_id = torch.tensor(word_to_ix[word_1], dtype=torch.long)
    w2_id = torch.tensor(word_to_ix[word_2], dtype=torch.long)
    w1_id = w1_id.to(device)
    w2_id = w2_id.to(device)
    
    word_1_vec = model.get_word_vector(w1_id)
    word_2_vec = model.get_word_vector(w2_id)
    
    # The norm of a vector (1D-matrix) is the square root of the sum of all the squared values within the vector.
    print(math.sqrt(torch.square(word_1_vec).sum()))    
    print(torch.linalg.norm(word_1_vec))
    print(torch.linalg.norm(word_2_vec))
    print(word_1_vec.dot(word_2_vec))
    
    word_distance = torch.linalg.norm(word_1_vec - word_2_vec)
    print("Distance between '{}' & '{}' : {:0.4f}".format(word_1, word_2, word_distance))
    word_similarity = (word_1_vec.dot(word_2_vec) / (torch.linalg.norm(word_1_vec) * torch.linalg.norm(word_2_vec)))
    print("Similarity between '{}' & '{}' : {:0.4f}".format(word_1, word_2, word_similarity))


In [16]:
similarity_cbow("neural", "network")

neural
network
4.28597472171565
tensor(4.2860, grad_fn=<CopyBackwards>)
tensor(3.1605, grad_fn=<CopyBackwards>)
tensor(2.5190, grad_fn=<DotBackward>)
Distance between 'neural' & 'network' : 4.8291
Similarity between 'neural' & 'network' : 0.1860


In [17]:
def predict_middle_word(prev_words, post_words):
    prev_words = prev_words.split()
    post_words = post_words.split()

    input_words= make_context_vector(prev_words + post_words, word_to_ix)
    input_words = input_words.to(device)
    output = model(input_words)
    out_ind = output.argmax(1)
#     print(word_to_ix)
#     out_word = word_to_ix.itos[out_ind.item()]
    out_word = key_list[val_list.index(out_ind.item())]
    print(out_word)

In [18]:
predict_middle_word("a recurrent", "network is")
predict_middle_word("LSTM is", "recurrent neural")

neural
a


### Now that you saw how to create the CBOW model (word2vec), you should work on doing the "opposite" model, Skip-Gram ###

Skip-gram as you saw on the lectures, reverses the problem so you need to predict through the "fake task" the context of the input

In [19]:
class Skipgram(nn.Module):
    def __init__(self, vocab_size, embedding_dim, contextSizeIn):
        super(Skipgram, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.linearFirst = nn.Linear(contextSizeIn*embedding_dim, 256)
        self.linearSecond = nn.Linear(256, vocab_size)
        self.contextSize = contextSizeIn
        #self.Vembeddings = nn.Embedding(vocab_sze, embedding_dim)
        #self.embedding_dim = embedding_dim
    def forward(self, inputs):
        embeddings = self.embedding(inputs).view((1,-1))
        linFirstOut = F.relu(self.linearFirst(embeddings))
        linSecondOut = self.linearSecond(linFirstOut)
        logProbabilities = F.log_softmax(linSecondOut, dim=1)#.view(contextSize, -1)
        return logProbabilities
    def get_word_vector(self, word):
        out = self.embedding(word)
        return out 
        

In [21]:
losses = []
lossFunction = nn.NLLLoss()
modelSKIPGRAM = Skipgram(vocab_size, EMBEDD_DIM, FULL_CONTEXT_SIZE)
optimizer = optim.SGD(modelSKIPGRAM.parameters(), lr=0.001)

lossesSKIPGRAM = train(modelSKIPGRAM, 100, data, optimizer, lossFunction)

modelSKIPGRAM.eval()
print(lossesSKIPGRAM)
#def train(model, epochs, data, optimizer, loss_fn):
#VOCAB_SIZE = len(vocab)
#EMBEDD_DIM = 10
#BATCH_SIZE = 6
#FULL_CONTEXT_SIZE = CONTEXT_SIZE * 2
#HIDDEN_SIZE = 256


[692.9156131744385, 686.4356069564819, 680.0392751693726, 673.7217481136322, 667.4763762950897, 661.301860332489, 655.1917119026184, 649.1386651992798, 643.1430008411407, 637.2004883289337, 631.30819272995, 625.4612672328949, 619.6552436351776, 613.8838677406311, 608.1488621234894, 602.4456775188446, 596.7697310447693, 591.1195185184479, 585.4914247989655, 579.8859539031982, 574.2986445426941, 568.7252094745636, 563.1637732982635, 557.6133276224136, 552.0697591304779, 546.5307179689407, 540.9942991733551, 535.4590595960617, 529.9211361408234, 524.380549788475, 518.8389794826508, 513.2940987348557, 507.74621045589447, 502.1934052705765, 496.6314307451248, 491.06397891044617, 485.4892771244049, 479.909029006958, 474.3190590143204, 468.71907168626785, 463.1129954457283, 457.4966977238655, 451.8700039386749, 446.2346438765526, 440.5888594985008, 434.93502074480057, 429.2773043513298, 423.610612988472, 417.94024670124054, 412.259856402874, 406.57705944776535, 400.89077639579773, 395.2040906

In [23]:
def predict_middle_word_SKIPGRAM(prev_words, post_words):
    prev_words = prev_words.split()
    post_words = post_words.split()

    input_words= make_context_vector(prev_words + post_words, word_to_ix)
    input_words = input_words.to(device)
    output = modelSKIPGRAM(input_words)
    out_ind = output.argmax(1)
#     print(word_to_ix)
#     out_word = word_to_ix.itos[out_ind.item()]
    out_word = key_list[val_list.index(out_ind.item())]
    print(out_word)

In [24]:
predict_middle_word_SKIPGRAM("a recurrent", "network is")
predict_middle_word_SKIPGRAM("LSTM is", "recurrent neural")

neural
a


In [25]:
def similarity_SKIPGRAM(word_1, word_2):
    
    # test word similarity
    print(word_1)
    print(word_2)
    w1_id = torch.tensor(word_to_ix[word_1], dtype=torch.long)
    w2_id = torch.tensor(word_to_ix[word_2], dtype=torch.long)
    w1_id = w1_id.to(device)
    w2_id = w2_id.to(device)
    
    word_1_vec = modelSKIPGRAM.get_word_vector(w1_id)
    word_2_vec = modelSKIPGRAM.get_word_vector(w2_id)
    
    # The norm of a vector (1D-matrix) is the square root of the sum of all the squared values within the vector.
    print(math.sqrt(torch.square(word_1_vec).sum()))    
    print(torch.linalg.norm(word_1_vec))
    print(torch.linalg.norm(word_2_vec))
    print(word_1_vec.dot(word_2_vec))
    
    word_distance = torch.linalg.norm(word_1_vec - word_2_vec)
    print("Distance between '{}' & '{}' : {:0.4f}".format(word_1, word_2, word_distance))
    word_similarity = (word_1_vec.dot(word_2_vec) / (torch.linalg.norm(word_1_vec) * torch.linalg.norm(word_2_vec)))
    print("Similarity between '{}' & '{}' : {:0.4f}".format(word_1, word_2, word_similarity))


In [26]:
similarity_SKIPGRAM("neural", "network")

neural
network
2.878335427742001
tensor(2.8783, grad_fn=<CopyBackwards>)
tensor(3.1281, grad_fn=<CopyBackwards>)
tensor(2.6805, grad_fn=<DotBackward>)
Distance between 'neural' & 'network' : 3.5649
Similarity between 'neural' & 'network' : 0.2977
